# Numpy Example of Graph Construction and Execution

## Graph Construction

Source nodes for the graph are created with the `source` function taking arguments an array of `Payload` for each node to be created and a list of dimension names. 

In [1]:
import numpy as np 
from cascade.fluent import Action, Payload, source

payloads = np.empty((4, 5), dtype=object)
payloads[:] = Payload(np.random.rand, [2, 3])
start = source(payloads, ["x", "y"])
start

Plotting the graph now, we will have 20 nodes

In [2]:
from cascade.graph.pyvis import to_pyvis

net_graph = to_pyvis(start.graph(), notebook=True)
net_graph.show("start.html")

With the functions such as `mean`, `std`, `minimum`, `maximum` etc we can reduce the array of nodes along a specified dimension
all the way down to a single node

In [3]:
single = start.mean("x").minimum("y")

From our initial payload in creating the source nodes, we chose a random array of shape (2, 3) inside each node. We can use `expand` to expose one of these internal dimensions into the array of nodes. To do this we need specify a new name for the dimension, its size and the axis of the internal array to take values from. After the operation, internally in each node we have arrays of shape (2,).

In [4]:
expanded = single.expand("z", 3, 1, 0)
expanded.nodes

<xarray.DataArray 'method@take:ee77f672679c18d0d2bc3393ead75ade67a97450f1edcf5703f8fe4c5b4dfe84' (
                                                                                                  z: 3)>
array([<Node 'method@take:ee77f672679c18d0d2bc3393ead75ade67a97450f1edcf5703f8fe4c5b4dfe84' at 0x14ccfe6d2c80>,
       <Node 'method@take:5ed43a47498b59df0a7b6ea0586023c20be9fc9bdf8a16f4e2f1d9618bb9ae87' at 0x14ccfe6d2e60>,
       <Node 'method@take:9b72230bc1edf6ebbfc94dddff9b5d7200c132e6ece03738291a51015c240185' at 0x14ccfe6d2830>],
      dtype=object)
Coordinates:
  * z        (z) int64 0 1 2

We can broadcast to match the shape of another existing set of nodes, which in this case creates duplicates of the single existing 
node along the z dimension. Note, this is an operation purely on the nodes of the graph and no operations are performed to the underlying arrays in each node.

In [5]:
single.broadcast(expanded).nodes

<xarray.DataArray (z: 3)>
array([<Node 'trivial:99b2121613b055fa52efb0222f2c430eade634eb3ca3e500c53f60c3fa16f139' at 0x14ccfe6d13f0>,
       <Node 'trivial:99b2121613b055fa52efb0222f2c430eade634eb3ca3e500c53f60c3fa16f139' at 0x14ccfe6d13f0>,
       <Node 'trivial:99b2121613b055fa52efb0222f2c430eade634eb3ca3e500c53f60c3fa16f139' at 0x14ccfe6d13f0>],
      dtype=object)
Coordinates:
  * z        (z) int64 0 1 2

### Low Level Operations

There are various low level functions `map`, `reduce`, and `transform` which allow the application of user-defined functions onto the array of nodes. The `map` operation applies a single payload to all nodes, or if a array of payloads is provided of the same shape as the array of nodes, then each node will get a unique payload applied to it.

In [6]:
# Single payload that scales the array in each node by 2
expanded.map(Payload(lambda x: x * 2)).nodes

<xarray.DataArray (z: 3)>
array([<Node '<lambda>:c65d7d1ec973cf25a3be05fa2259cdc5903bd4deb6578a4e3524fd1dc8c62214' at 0x14ccfe6d2500>,
       <Node '<lambda>:51c565a8ffb480d662b3f495390de0f59f8dc30d51f6da1212184c81a526dd20' at 0x14ccfe6d1420>,
       <Node '<lambda>:6f6b7a2d463545d34b9f9dea9f77abe22cd87282d94d5744843047af3cf481c9' at 0x14ccfe6d1330>],
      dtype=object)
Coordinates:
  * z        (z) int64 0 1 2

In [7]:
# Or we can scale the array in each node by a different value 
mapped = expanded.map([Payload(lambda x, a: x * a) for x in range(1, 4)])

Arbitrary reduction operations can be applied with the `reduce` operation, which takes arguments `Payload` and the name of the dimension the reduction should be performed along. If no dimension name is supplied, the reduction is performed along the first axis. The higher level functions `mean`, `std`, `minimum`, `maximum` are all `reduce` operations with a pre-defined payload.

Finally, we have `transform` which allows the shape of the array of nodes in the subsequent action to be changed. The operation takes 
- a function of the form `func(action: Action, arg: Any) -> Action`
- a list of values for `arg` 
- a name for new dimension along which `arg` varies

The resulting nodes will be output of `func` with the different values of `arg`.

In [8]:
def _example_transform(action: Action, threshold: float) -> Action:
    ret = action.map(Payload(lambda x: x if x > threshold else 0))
    ret._add_dimension("threshold", threshold)
    return ret

mapped.transform(_example_transform, [0, 1, 10], "threshold").nodes

<xarray.DataArray (threshold: 3, z: 3)>
array([[<Node '<lambda>:1dd88367485bd0b869081d0a24be58784f44a22e87768f82083bbf9046732192' at 0x14ccfe6a3fa0>,
        <Node '<lambda>:f62478b300817edaaf361f22ac12c49442ebabc07b94e5ee6b096258a9ac6cf1' at 0x14ccfe6a3f40>,
        <Node '<lambda>:81914c94e1bd218591d00efee2815824892f0c607d1b5c181a2e8eb9e5587440' at 0x14ccfe6a15d0>],
       [<Node '<lambda>:e0464917965e6a62610397dfdea76fd49780e6868664e282e04a4c2181e889e0' at 0x14ccfe6a3640>,
        <Node '<lambda>:7b1530509a66e58a54ad526ab307721f2e80ae55dd1738c48a085eaa9d5cf5ba' at 0x14ccfe6a3610>,
        <Node '<lambda>:70581f3565ae0610246438facedf60575782f15e645fc16a5a0365f73ac50335' at 0x14ccfe6a3580>],
       [<Node '<lambda>:699bbfb1a997df2e5a74677d6793a54b45d36e8f9cd154e69f95e35dc14a9e2d' at 0x14ccfe6a3c70>,
        <Node '<lambda>:4719f34caa070aa76085a176f64364c7ba2478680876a7ad97a13e6d7c8fdbb2' at 0x14ccfe6a12d0>,
        <Node '<lambda>:8088d9f1c739d9281ed71c63c3b45f83e7a820d56076848d69546cdccd1f3f7e' at 0x14ccfe6a1180>]],
      dtype=object)
Coordinates:
  * threshold  (threshold) int64 0 1 10
  * z          (z) int64 0 1 2

## Graph Execution

Example graph composed by combining the various operations detailed in the Graph Construction section

In [9]:
import numpy as np 
from cascade.fluent import Payload, source

payloads = np.empty((4, 5), dtype=object)
payloads[:] = Payload(np.random.rand, [2, 3])
graph = (
    source(payloads, ["x", "y"])
    .mean("x")
    .minimum("y")
    .expand("z", 3, 1, 0)
    .map([Payload(lambda x, a=a: x * a) for a in range(1, 4)])
    .map(Payload(lambda x: print("RESULT", x)))
    .graph()
)

At the moment, the graphs can be executed using Dask but the Dask dynamic scheduling.

In [10]:
import os
from cascade.executor import DaskExecutor
from cascade.scheduler import Schedule

os.environ["DASK_LOGGING__DISTRIBUTED"]="debug"
schedule = Schedule(graph, None, {})
executor = DaskExecutor(schedule)
executor.execute(memory_limit="10GB", n_workers=1, threads_per_worker=1)

2023-12-07 14:40:31,559 - distributed.system - DEBUG - Setting system memory limit based on cgroup value defined in /sys/fs/cgroup/memory/memory.soft_limit_in_bytes


RESULT [1.46478444 0.65478046]
RESULT [0.59523715 2.82901838]
RESULT [0.56622669 0.81399934]


2023-12-07 14:40:32,151 - distributed.worker - DEBUG - Attempted to close worker that is already Status.closed. Reason: worker-close
